# NORMAL Dataset Formatter (Protocol v1.3.2-NORMAL)

This notebook **only** processes the `DataSet/NORMAL/` folder because NORMAL files have a different schema from fault folders.

## Input
- `DataSet/NORMAL/*.csv`

## Output
- Useable: `ProcessedData/NORMAL/NORMAL_Data_<N>.csv`
- Not useable (FLAG=1 present): `ProcessedData/NOT_USEABLE/NORMAL/NORMAL_Data_<N>.csv`

## Protocol Output Columns (Strict)
`DFDT,FREQ,IA_MAG,IA_ANG,IB_MAG,IB_ANG,IC_MAG,IC_ANG,VA_MAG,VA_ANG,VB_MAG,VB_ANG,VC_MAG,VC_ANG,TIMESTAMP,ERROR_CODE`

### Rule: FLAG=1 → NOT USEABLE
If **any row** in the source file has `FLAG == 1`, the **entire file** is written into `ProcessedData/NOT_USEABLE/NORMAL/`.

Additionally, rows with `FLAG==1` are marked as `ERROR_CODE = 359` (UNUSABLE_SAMPLE). Otherwise `ERROR_CODE = 000`.

> Edit `ROOT` to your Thesis root folder: `/Users/yuki/Desktop/Thesis`


In [1]:
# Cell 1 — Imports & Paths
from __future__ import annotations

from pathlib import Path
import re
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)

# ====== USER EDIT HERE ======
ROOT = Path("/Users/yuki/Desktop/Thesis")  # <- Thesis root
# ============================

DATASET_ROOT = ROOT / "DataSet" / "NORMAL"
OUTPUT_ROOT  = ROOT / "ProcessedData"
OUT_OK_DIR   = OUTPUT_ROOT / "NORMAL"
OUT_BAD_DIR  = OUTPUT_ROOT / "NOT_USEABLE" / "NORMAL"

PROTOCOL_COLUMNS = [
    "DFDT","FREQ",
    "IA_MAG","IA_ANG","IB_MAG","IB_ANG","IC_MAG","IC_ANG",
    "VA_MAG","VA_ANG","VB_MAG","VB_ANG","VC_MAG","VC_ANG",
    "TIMESTAMP","ERROR_CODE"
]

# Protocol codes
NORMAL_CODE = 0          # 000
UNUSABLE_CODE = 359      # UNUSABLE_SAMPLE
FLAG_ERROR_CODE = 151    # FLAG_ERROR (if you want to log it)

def ensure_dir(p: Path) -> None:
    p.mkdir(parents=True, exist_ok=True)

ensure_dir(OUT_OK_DIR)
ensure_dir(OUT_BAD_DIR)


In [2]:
# Cell 2 — Helpers

def seconds_to_mmss_s(t: float, decimals: int = 1) -> str:
    """Convert seconds to MM:SS.s."""
    if pd.isna(t):
        return ""
    t = float(t)
    mm = int(t // 60)
    ss = t - 60 * mm
    return f"{mm:02d}:{ss:0{2+1+decimals}.{decimals}f}"


def parse_normal_schema_columns(df: pd.DataFrame) -> dict[str, str]:
    """Locate required columns in NORMAL file by regex."""
    cols = df.columns.astype(str).tolist()

    def pick(patterns: list[str]) -> str | None:
        for pat in patterns:
            rx = re.compile(pat)
            for c in cols:
                if rx.search(c):
                    return c
        return None

    mapping = {
        "Timestamp": pick([r"^Timestamp$"]),
        "FREQ": pick([r":FREQ\b", r"\bFREQ\b"]),
        "DFDT": pick([r":DFDT\b", r"\bDFDT\b"]),
        "FLAG": pick([r":FLAG\b", r"\bFLAG\b"]),
        # Voltages L1/L2/L3
        "VA_MAG": pick([r"-L1:MAG\b", r"\bL1:MAG\b"]),
        "VA_ANG": pick([r"-L1:ANG\b", r"\bL1:ANG\b"]),
        "VB_MAG": pick([r"-L2:MAG\b", r"\bL2:MAG\b"]),
        "VB_ANG": pick([r"-L2:ANG\b", r"\bL2:ANG\b"]),
        "VC_MAG": pick([r"-L3:MAG\b", r"\bL3:MAG\b"]),
        "VC_ANG": pick([r"-L3:ANG\b", r"\bL3:ANG\b"]),
        # Currents C1/C2/C3
        "IA_MAG": pick([r"-C1:MAG\b", r"\bC1:MAG\b"]),
        "IA_ANG": pick([r"-C1:ANG\b", r"\bC1:ANG\b"]),
        "IB_MAG": pick([r"-C2:MAG\b", r"\bC2:MAG\b"]),
        "IB_ANG": pick([r"-C2:ANG\b", r"\bC2:ANG\b"]),
        "IC_MAG": pick([r"-C3:MAG\b", r"\bC3:MAG\b"]),
        "IC_ANG": pick([r"-C3:ANG\b", r"\bC3:ANG\b"]),
    }
    missing = [k for k,v in mapping.items() if v is None and k not in ("FLAG",)]
    if missing:
        raise ValueError(f"Missing required columns: {missing}\nAvailable: {cols}")
    return mapping


def enforce_nan_and_numeric(out: pd.DataFrame) -> pd.DataFrame:
    """Force numeric columns to numeric; bad/empty -> NaN; write NaN as literal later."""
    out = out.copy()
    numeric_cols = [c for c in PROTOCOL_COLUMNS if c not in ("TIMESTAMP", "ERROR_CODE")]
    for c in numeric_cols:
        out[c] = pd.to_numeric(out[c], errors="coerce")
    out["ERROR_CODE"] = pd.to_numeric(out["ERROR_CODE"], errors="coerce").fillna(UNUSABLE_CODE).astype(int)
    out["TIMESTAMP"] = out["TIMESTAMP"].astype(str)
    return out


def build_protocol_from_normal(df: pd.DataFrame) -> tuple[pd.DataFrame, bool]:
    """Return (protocol_df, has_flag_one)."""
    m = parse_normal_schema_columns(df)

    # DFDT/FREQ
    out = pd.DataFrame({
        "DFDT": df[m["DFDT"]],
        "FREQ": df[m["FREQ"]],

        "IA_MAG": df[m["IA_MAG"]],
        "IA_ANG": df[m["IA_ANG"]],
        "IB_MAG": df[m["IB_MAG"]],
        "IB_ANG": df[m["IB_ANG"]],
        "IC_MAG": df[m["IC_MAG"]],
        "IC_ANG": df[m["IC_ANG"]],

        "VA_MAG": df[m["VA_MAG"]],
        "VA_ANG": df[m["VA_ANG"]],
        "VB_MAG": df[m["VB_MAG"]],
        "VB_ANG": df[m["VB_ANG"]],
        "VC_MAG": df[m["VC_MAG"]],
        "VC_ANG": df[m["VC_ANG"]],
    })

    # Timestamp conversion:
    # Source Timestamp is full datetime; we convert to relative time from first sample.
    ts = pd.to_datetime(df[m["Timestamp"]], errors="coerce")
    t0 = ts.iloc[0]
    rel_sec = (ts - t0).dt.total_seconds()
    out["TIMESTAMP"] = rel_sec.apply(lambda x: seconds_to_mmss_s(x, decimals=1))

    # FLAG rule
    flag = pd.to_numeric(df[m["FLAG"]], errors="coerce").fillna(0) if m["FLAG"] else pd.Series([0]*len(df))
    has_flag_one = bool((flag == 1).any())

    # ERROR_CODE per row
    out["ERROR_CODE"] = NORMAL_CODE
    out.loc[flag == 1, "ERROR_CODE"] = UNUSABLE_CODE

    out = enforce_nan_and_numeric(out)
    out = out[PROTOCOL_COLUMNS]
    return out, has_flag_one


In [3]:
# Cell 3 — Batch Process NORMAL Only (separate pipeline)

normal_csvs = sorted(DATASET_ROOT.glob("*.csv"))
print(f"Found {len(normal_csvs)} NORMAL csv files under: {DATASET_ROOT}")

ok_written = []
bad_written = []
failed = []

counter_ok = 0
counter_bad = 0

for p in normal_csvs:
    try:
        df = pd.read_csv(p)
        out, has_flag_one = build_protocol_from_normal(df)

        if has_flag_one:
            counter_bad += 1
            out_path = OUT_BAD_DIR / f"NORMAL_Data_{counter_bad}.csv"
            out.to_csv(out_path, index=False, na_rep="NaN")
            bad_written.append(out_path)
        else:
            counter_ok += 1
            out_path = OUT_OK_DIR / f"NORMAL_Data_{counter_ok}.csv"
            out.to_csv(out_path, index=False, na_rep="NaN")
            ok_written.append(out_path)

    except Exception as e:
        failed.append((p, str(e)))

print(f"OK: {len(ok_written)}")
print(f"NOT_USEABLE (FLAG=1): {len(bad_written)}")
print(f"FAILED: {len(failed)}")

if failed:
    print("\nFirst 5 failures:")
    for p, msg in failed[:5]:
        print("-", p, "->", msg)

(ok_written[:3], bad_written[:3])


Found 10 NORMAL csv files under: /Users/yuki/Desktop/Thesis/DataSet/NORMAL
OK: 10
NOT_USEABLE (FLAG=1): 0
FAILED: 0


([PosixPath('/Users/yuki/Desktop/Thesis/ProcessedData/NORMAL/NORMAL_Data_1.csv'),
  PosixPath('/Users/yuki/Desktop/Thesis/ProcessedData/NORMAL/NORMAL_Data_2.csv'),
  PosixPath('/Users/yuki/Desktop/Thesis/ProcessedData/NORMAL/NORMAL_Data_3.csv')],
 [])

In [4]:
# Cell 4 — Spot Check

import random

def show_head(path: Path, n: int = 8):
    print("File:", path)
    df = pd.read_csv(path, keep_default_na=False)
    display(df.head(n))

if ok_written:
    show_head(random.choice(ok_written))

if bad_written:
    show_head(random.choice(bad_written))


File: /Users/yuki/Desktop/Thesis/ProcessedData/NORMAL/NORMAL_Data_3.csv


,DFDT,FREQ,IA_MAG,IA_ANG,IB_MAG,IB_ANG,IC_MAG,IC_ANG,VA_MAG,VA_ANG,VB_MAG,VB_ANG,VC_MAG,VC_ANG,TIMESTAMP,ERROR_CODE
0,8.734326e-39,49.97304,7.822668,138.1225,8.691373,-106.2133,8.392998,21.77386,237.0452,51.11538,238.9399,170.9902,238.4278,-68.55167,00:00.0,0
1,2.136978e-38,49.97654,7.810932,138.1991,8.662885,-106.8164,8.384695,21.62393,237.1305,51.04172,238.8390,170.8890,238.5136,-68.67872,00:00.0,0
2,6.980124e-39,49.97149,7.800079,137.8496,8.706089,-106.4714,8.380802,21.82910,237.0893,50.92616,238.9330,170.8060,238.4253,-68.75208,00:00.0,0
3,1.831768e-38,49.98112,7.842213,138.1579,8.665588,-106.2996,8.400892,21.59436,237.1417,50.88297,238.8562,170.7228,238.5508,-68.84180,00:00.0,0
4,4.271578e-40,49.97788,7.836653,137.9353,8.684920,-106.3470,8.410437,21.50559,237.0627,50.79112,238.9786,170.6616,238.4636,-68.87663,00:00.0,0
5,1.437988e-38,49.98080,7.790283,137.9093,8.682239,-106.9898,8.403098,21.30512,237.1379,50.72907,238.8589,170.5818,238.5105,-68.98602,00:00.1,0
6,1.857077e-38,49.97363,7.777824,137.7387,8.704532,-106.8511,8.362633,21.32487,237.0730,50.61716,238.9592,170.4970,238.4361,-69.05197,00:00.1,0
7,8.174524e-39,49.97876,7.814526,138.0160,8.663404,-106.7562,8.411530,21.13941,237.1338,50.55600,238.8644,170.4112,238.4976,-69.15806,00:00.1,0
